In [6]:
# 🧠 U-Net Bottleneck Feature Map Visualization (Animated GIF Style)

import os
import sys
import torch
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import imageio
from IPython.display import Image, display

# 🔧 Add src to Python path
project_root = r"F:\Projects\ML_Models\BrainTome"
sys.path.append(project_root)

# 📂 Paths - Customize for your system
MODEL_PATH = r"F:\Projects\ML_Models\BrainTome\results\models\best_model.pth"
DATA_DIR = r"F:\Projects\ML_Models\BrainTome\patch_dataset\BraTS-GLI-00025-000"
GIF_OUTPUT_PATH = r"F:\Projects\ML_Models\BrainTome\results\feature_maps\bottleneck_feature_maps.gif"

# ✅ Create result folder if it doesn't exist
os.makedirs(os.path.dirname(GIF_OUTPUT_PATH), exist_ok=True)

# ✅ Import model
from src.model import UNet3D

# 🚀 Load trained model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = UNet3D(in_channels=4, out_channels=1).to(device)
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.eval()

# 📡 Register a hook on bottleneck
feature_maps = []

def hook_fn(module, input, output):
    feature_maps.append(output.detach().cpu().numpy())

hook = model.bottleneck.register_forward_hook(hook_fn)

# 📦 Load a sample patch
X_path = os.path.join(DATA_DIR, "X_0.npy")
X_patch = np.load(X_path)  # Shape: (4, 64, 64, 64)
X_tensor = torch.tensor(X_patch).unsqueeze(0).to(device).float()

# 🔍 Forward pass to extract feature maps
with torch.no_grad():
    _ = model(X_tensor)

# 🎯 Process the bottleneck feature maps
feat = feature_maps[0][0]  # shape: (num_channels, D, H, W)
print(f"[INFO] Feature map shape: {feat.shape}")

# 🎞️ Convert channels to 2D slices and make frames
frames = []
for i in range(feat.shape[0]):
    slice_2d = feat[i, :, :, feat.shape[3] // 2]  # Mid-axial slice
    normed = (slice_2d - np.min(slice_2d)) / (np.ptp(slice_2d) + 1e-8)
    img = (normed * 255).astype(np.uint8)
    frames.append(img)

# 🎬 Save animated GIF
imageio.mimsave(GIF_OUTPUT_PATH, frames, duration=0.12)
print(f"✅ GIF saved to: {GIF_OUTPUT_PATH}")

# 🖼️ Display the result
display(Image(GIF_OUTPUT_PATH))


FileNotFoundError: [Errno 2] No such file or directory: 'F:\\Projects\\ML_Models\\BrainTome\\results\\models\\best_model.pth'